In [13]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
print(np.__version__)

1.16.5


In [14]:
path = '/Users/hph/cmzoom_catalog/catalog_acc/'
targetid='_rms3e6_k14_dv3_dd1_dp17_pp6_pm2_gal_10-23-19.fits'
rtable = Table.read(path+'master_tab'+targetid)
region_mask = fits.open('/Users/hph/cmzoom_catalog/Continuum/CMZoom_Mask_new.fits')
mask_data = region_mask[0].data

In [3]:
rtable.add_column(np.zeros_like(rtable['mass'],dtype='str'), name='leaf_ID')

In [4]:
names = np.array(['G1.683-0.089', 
                      'G1.670-0.130', 
                      'G1.651-0.050', 
                      'G1.602+0.018',
                      'G1.127-0.106',
                      'G1.085-0.027', 
                      'G1.038-0.074', 
                      'G0.891-0.048',
                      'G0.714-0.100',
                      'G0.699-0.028',
                      'G0.619+0.012',
                      'G0.489+0.010',
                      'G0.412+0.052',
                      'G0.393-0.034',
                      'G0.380+0.050',
                      'G0.340+0.055',
                      'G0.326-0.085',
                      'G0.316-0.201',
                      'G0.253+0.016',
                      'G0.212-0.001',
                      'G0.145-0.086',
                      'G0.106-0.082',
                      'G0.070-0.035',
                      'G0.068-0.075',
                      'G0.054+0.027',
                      'G0.014+0.021',
                      'G0.001-0.058',
                      'G359.948-0.052',
                      'G359.889-0.093',
                      'G359.865+0.022',
                      'G359.863-0.069',
                      'G359.734+0.002',
                      'G359.648-0.133',
                      'G359.611+0.018',
                      'G359.615-0.243',
                      'G359.484-0.132',
                      'G359.137+0.031'])

In [5]:
w = wcs.WCS(path+'CMZoom_continuum_pbcor.fits').celestial

xglon = w.wcs_world2pix(rtable['glon'],rtable['glat'],0)[0]
yglat = w.wcs_world2pix(rtable['glon'],rtable['glat'],0)[1]

In [7]:
abc = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
       'o','p','q','r','s','t','u','v','w','x','y','z','za','zb',
       'zc','zd','ze','zf','zg','zh','zi','zj','zk','zl','zm','zn','zo','zp']

In [8]:
problem_list=np.empty(0)
cloud_id = np.empty(len(rtable['mass']),dtype=object)
for i in range(len(cloud_id)):
    cnum = int(mask_data[int(yglat[i])][int(xglon[i])])
    if(cnum==1000):
          np.append(problem_list,i)
    else:    
        cloud_id[i]=str(names[int(mask_data[int(yglat[i])][int(xglon[i])])-1])
### handle special case:
cloud_id[36]=names[0]

leaf_ID = cloud_id.copy()
### append mass ordered label
for i in range(len(cloud_id)):
    cmasses=np.flip(np.sort(rtable['mass'][np.where(cloud_id==cloud_id[i])]))
    leaf_ID[i]=cloud_id[i]+abc[np.where(cmasses==rtable['mass'][i])[0][0]]

In [10]:
rtable['leaf_ID']=leaf_ID.copy().astype(str)


In [12]:
rtable.write('/Users/hph/cmzoom_catalog/catalog_acc/catalog_acc.fits', 
             format='fits', overwrite=True)